In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from evaluation.utils import entropy_batch_mixing, knn_purity
import scanpy as sc
import seaborn as sns
import scIB as scib
import pandas as pd
from matplotlib import pyplot as plt

In [2]:
models= ['Harmony', 'Liger','Scanorama','Seurat']
datasets = ['brain']
versions = [1]
ratios = [1]

In [3]:
def compute_metrics(latent_adata, adata, model, dataset, rqr=None, batch_key='study', label_key='cell_type'):
    latent_adata.obsm['X_pca'] = latent_adata.X
    print(adata.shape, latent_adata.shape)
    n_batches = len(adata.obs[batch_key].unique().tolist())
    
    scores = scib.metrics.metrics(adata, latent_adata, batch_key, label_key, 
                                  nmi_=True, ari_=True, silhouette_=True, pcr_=True, graph_conn_=True, 
                                  isolated_labels_=True, hvg_score_=False)
    scores = scores.T
    scores = scores[['NMI_cluster/label', 'ARI_cluster/label', 'ASW_label', 'ASW_label/batch', 
                     'PCR_batch', 'isolated_label_F1', 'isolated_label_silhouette', 'graph_conn']]
    
    ebm = entropy_batch_mixing(latent_adata, batch_key, n_neighbors=15)
    knn = knn_purity(latent_adata, label_key, n_neighbors=15)
    
    scores['EBM'] = ebm
    scores['KNN'] = knn
    scores['method'] = model
    scores['data'] = dataset
    scores['rqr'] = rqr
    scores.rqr = scores.rqr.round(2)
    scores['reference_time'] = 0.0
    scores['query_time'] = 0.0
    
    return scores

## Calculating metrics for all ratios:

In [4]:
for dataset in datasets:   
    if dataset == 'pbmc':
        adata = sc.read(os.path.expanduser(f'~/Documents/benchmarking_datasets/Immune_ALL_human_wo_villani_rqr_normalized_hvg.h5ad'))
        batch_key = 'condition'
        label_key = 'final_annotation'
    elif dataset == 'brain':
        adata = sc.read(os.path.expanduser(f'~/Documents/benchmarking_datasets/mouse_brain_subsampled_normalized_hvg.h5ad'))
        batch_key = 'study'
        label_key = 'cell_type'
    elif dataset == 'pancreas':
        adata = sc.read(os.path.expanduser(f'~/Documents/benchmarking_datasets/pancreas_normalized.h5ad'))
        batch_key = 'study'
        label_key = 'cell_type'
    for version in versions:
        scores = None
        for model in models:
            for ratio in ratios:
                latent_adata = sc.read(os.path.expanduser(f'~/Documents/matin/{dataset}/{model}/result_adata.h5ad'))
                df = compute_metrics(latent_adata, adata, model, dataset, 1, batch_key, label_key)
                scores = pd.concat([scores, df], axis=0) if scores is not None else df
        scores.to_csv(os.path.expanduser(f'~/Documents/matin/{dataset}_matin_models.csv'), index=False)

(332129, 4000) (332129, 20)
clustering...
NMI...
ARI...
silhouette score...
PC regression...
isolated labels...
Graph connectivity...


/home/marco/.pyenv/versions/scarches_bench/lib/python3.7/site-packages/scIB/metrics.py:1746: FutureWarning: This location for 'connectivities' is deprecated. It has been moved to .obsp[connectivities], and will not be accesible here in a future version of anndata.
  _,labs = connected_components(adata_post_sub.uns['neighbors']['connectivities'], connection='strong')


Calculating EBM with n_cat = 4
EBM: 0.2319989896198924
KNN-P: 0.9175232457209381
(332129, 4000) (332129, 20)
clustering...
NMI...
ARI...
silhouette score...
PC regression...
isolated labels...
Graph connectivity...


/home/marco/.pyenv/versions/scarches_bench/lib/python3.7/site-packages/scIB/metrics.py:1746: FutureWarning: This location for 'connectivities' is deprecated. It has been moved to .obsp[connectivities], and will not be accesible here in a future version of anndata.
  _,labs = connected_components(adata_post_sub.uns['neighbors']['connectivities'], connection='strong')


Calculating EBM with n_cat = 4
EBM: 0.1255170419364338
KNN-P: 0.909076479331112
(332129, 4000) (332129, 20)
clustering...
NMI...
ARI...
silhouette score...
PC regression...
Variance contribution increased after integration!
Setting PCR comparison score to 0.
isolated labels...
Graph connectivity...
Calculating EBM with n_cat = 4


/home/marco/.pyenv/versions/scarches_bench/lib/python3.7/site-packages/scIB/metrics.py:1746: FutureWarning: This location for 'connectivities' is deprecated. It has been moved to .obsp[connectivities], and will not be accesible here in a future version of anndata.
  _,labs = connected_components(adata_post_sub.uns['neighbors']['connectivities'], connection='strong')


EBM: 0.1390333334542057
KNN-P: 0.9071386417566067
(332129, 4000) (332129, 20)
clustering...
NMI...
ARI...
silhouette score...
PC regression...
isolated labels...
Graph connectivity...


/home/marco/.pyenv/versions/scarches_bench/lib/python3.7/site-packages/scIB/metrics.py:1746: FutureWarning: This location for 'connectivities' is deprecated. It has been moved to .obsp[connectivities], and will not be accesible here in a future version of anndata.
  _,labs = connected_components(adata_post_sub.uns['neighbors']['connectivities'], connection='strong')


Calculating EBM with n_cat = 4
EBM: 0.3169109995786411
KNN-P: 0.889555347178668
